In [ ]:
%cd ../

from src.requirements import *
from src.utils import *
%matplotlib inline

## ******************************************************************************************************************************
## OPEN CONFIG FILE
## ******************************************************************************************************************************

with open('configFile.json') as json_data_file:
    configFile = json.load(json_data_file)

## ******************************************************************************************************************************
## SET BIG QUERY CREDENTIALS
## ******************************************************************************************************************************

SERVICE_ACCOUNT_FILE = configFile['config']['BQ_key_file']
client_bq = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)
bq_dataset_id = configFile['config']['bq_dataset']
bq_dataset_ref = client_bq.dataset(bq_dataset_id)

## ******************************************************************************************************************************
## CARTO
## ******************************************************************************************************************************

carto_username = configFile['config']['CARTO']['username']
carto_API = configFile['config']['CARTO']['API_key']

creds = Credentials(carto_username, carto_API)
set_default_credentials(creds)

%cd etl/

## Join Census county-level data by aggregating block group-level data

### Get aggregation method

In [ ]:
agg_method_demo, dens_method_demo, vars_demo, methods_table_demo = get_method('carto-do.ags.demographics_sociodemographics_usa_blockgroup_2015_yearly_2020', 'b','a')
agg_method_bus, dens_method_bus, vars_bus, methods_table_bus = get_method('carto-do.ags.demographics_businesscounts_usa_blockgroup_2015_yearly_2020', 'c','a')

#print(agg_method_demo)

In [ ]:
var_cols = vars_demo + vars_bus
var_cols = [i for i in var_cols if i.replace('_dens','') not in ['POPCY',
                                                'POPPY',
                                                'HHDPY',
                                                'DWLPY',
                                                'AGEPYMED',
                                                'INCPYPCAP',
                                                'INCPYAVEHH',
                                                'INCPYMEDHH',
                                                'POPCYGRP',
                                                'POPCYGRPI',
                                                'SEXCYMAL',
                                                'SEXCYFEM',
                                                'AGECY0004',
                                                'AGECY0509',
                                                'AGECY1014',
                                                'AGECY1519',
                                                'AGECY2024',
                                                'AGECY2529',
                                                'AGECY3034',
                                                'AGECY3539',
                                                'AGECY4044',
                                                'AGECY4549',
                                                'AGECY5054',
                                                'AGECY5559',
                                                'AGECY6064',
                                                'AGECY6569',
                                                'AGECY7074',
                                                'AGECY7579',
                                                'AGECY8084',
                                                'HINCYLT10',
                                                'HINCY1015',
                                                'HINCY1520',
                                                'HINCY2025',
                                                'HINCY2530',
                                                'HINCY3035',
                                                'HINCY3540',
                                                'HINCY4045',
                                                'HINCY4550',
                                                'HINCY5060',
                                                'HINCY6075',
                                                'HINCY75100',
                                                'HINCY10025',
                                                'HINCY12550',
                                                'HINCY15020',
                                                'HINCYGT200',
                                                'HINCYMED24',
                                                'HINCYMED25',
                                                'HINCYMED35',
                                                'HINCYMED45',
                                                'HINCYMED55',
                                                'HINCYMED65',
                                                'HINCYMED75',
                                                'ESZCYA',
                                                'ESZCYB',
                                                'ESZCYC',
                                                'ESZCYD',
                                                'ESZCYE',
                                                'ESZCYF',
                                                'ESZCYG',
                                                'ESZCYH',
                                                'ESZCYI',
                                                'ESCCYMINE',
                                                'ESCCYCONST',
                                                'ESCCYSCONS',
                                                'ESCCYTECH',
                                                'ESCCYUTILS',
                                                'ESCCYWHOLI',
                                                'ESCCYWAREH',
                                                'ESCCYGENIN',
                                                'ESCCYMFGHV',
                                                'ESCCYMFGGN',
                                                'ESCCYMFGLT',
                                                'ESCCYMFGSM',
                                                'ESCCYRTL',
                                                'ESCCYREST',
                                                'ESCCYPRSRP',
                                                'ESCCYAUTOR',
                                                'ESCCYHOTEL',
                                                'ESCCYTHEAT',
                                                'ESCCYEQPRN',
                                                'ESCCYWHOLC',
                                                'ESCCYGENCM',
                                                'ESCCYADMN',
                                                'ESCCYFIRE',
                                                'ESCCYFIRES',
                                                'ESCCYPROF',
                                                'ESCCYBUSS',
                                                'ESCCYOFF',
                                                'ESCCYMED',
                                                'ESCCYSCL',
                                                'ESCCYLIB',
                                                'ESCCYHOSP',
                                                'ESCCYMUS',
                                                'ESCCYOUTRC',
                                                'ESCCYPUBAD',
                                                'ESCCYCHURC',
                                                'ESCCYOTHER',
                                                'ESCCYAGPRO',
                                                'ESCCYAGSRV',
                                                'AGECYGT85',
                                                'AGECYGT25',
                                                'AGECYGT15',
                                                'LBFCYPOP16']]

In [ ]:
methods_table_demo[methods_table_demo.column_name.isin([x.replace('_dens','') for x in var_cols])][['query_string_vars','description','agg_method']].to_csv('../data/' + filename + '_vars_demo.csv', index = False)

In [ ]:
methods_table_bus[methods_table_bus.column_name.isin([x.replace('_dens','') for x in var_cols])][['query_string_vars','description','agg_method']].to_csv('../data/' + filename + '_vars_bus.csv', index = False)

### Join in Google Bigquery the table with Google mobility data and COVID-19 cases and the aggregated county-level Census data 

In [ ]:
table_name = '{}.{}.epi_mobility_census_county_full'.format(bq_dataset_ref.project,bq_dataset_id)
client_bq.delete_table(table_name, not_found_ok=True)  
q = """
    CREATE TABLE `{}` AS
        WITH tmp1 AS 
        (
          SELECT DISTINCT 
              a.sub_region_2, 
              {},
              {},
          FROM `{project_id}.{dataset_id}.epi_mobility_county` a
          JOIN `carto-do.ags.demographics_sociodemographics_usa_blockgroup_2015_yearly_2019` b
          on a.geoid = SUBSTR(b.geoid, 1, 5)
          JOIN `carto-do.ags.demographics_businesscounts_usa_blockgroup_2015_yearly_2019` c
          on b.geoid = c.geoid
          JOIN `carto-do.ags.geography_usa_blockgroup_2015` d
          on b.geoid = d.geoid
        GROUP BY sub_region_2
    )
    SELECT b.*,
            {},
            {}
    FROM `{project_id}.{dataset_id}.epi_mobility_county` b
    LEFT JOIN tmp1 a
    ON a.sub_region_2=b.sub_region_2
""".format(table_name,
           agg_method_demo,
           agg_method_bus,
           dens_method_demo,
           dens_method_bus,
           project_id = bq_dataset_ref.project,
           dataset_id = bq_dataset_id
          )
#print(q)           
df = client_bq.query(q).to_dataframe()